In [1]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
def find_pairs(words, wanted = 'narodnaya'):
    pairs = []
    for postition, word in enumerate(words[:-1]):
        if word == wanted:
            pairs.append(((words[postition]+"_"+words[postition+1]),1))
    return pairs

In [2]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
wiki_lower = wiki.map(lambda words: [x.lower() for x in words])
wiki_pairs = wiki_lower.flatMap(lambda x: find_pairs(x, 'narodnaya')).reduceByKey(lambda x,y:x+y).sortByKey()

In [3]:
result = wiki_pairs.collect()
for pair, count in result:
    print '{}\t{}'.format(pair, count)

narodnaya_gazeta	1
narodnaya_volya	9
